<a href="https://colab.research.google.com/github/gichurejoy/Machine-Learning/blob/main/cat2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk -q

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.0 MB/s 
     |████████████████████████████████| 115 kB 46.0 MB/s 
     |████████████████████████████████| 163 kB 53.7 MB/s 
     |████████████████████████████████| 212 kB 51.3 MB/s 
     |████████████████████████████████| 95 kB 4.9 MB/s 
     |████████████████████████████████| 127 kB 56.0 MB/s 
     |████████████████████████████████| 115 kB 69.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [4]:
from datasets import load_dataset
dataset = load_dataset("swahili_news")


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 22207
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7338
    })
})

In [8]:
import pandas as pd
import numpy as np


In [46]:
import warnings
warnings.filterwarnings('ignore')
dataset = load_dataset("swahili_news")
train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]
df = pd.DataFrame()
df["text"] = train_texts + test_texts
df["label"] = train_labels + test_labels
df["text"].iloc[np.random.choice(df.index, 100, replace=False)] = np.nan
df = df.sample(np.random.randint(24000, 24500))
df = df.reset_index(drop=True)
print("Your final dataset has have {} rows".format(df.shape[0]))
print("You have the following value counts for the label column:")
print(df["label"].value_counts())
df.head()


  0%|          | 0/2 [00:00<?, ?it/s]

Your final dataset has have 24005 rows
You have the following value counts for the label column:
1    10948
2     6490
4     2367
3     2005
0     1405
5      790
Name: label, dtype: int64


,text,label
0,Mchezo huo wa pekee ambao ni maalumu kwa mich...,2
1,['Mkufunzi wa Tottenham Jose Mourinho amefufua...,2
2,MWIMBAJI nguli wa muziki wa injili Afrika Mash...,4
3,MKURUGENZI Mkuu wa Mamlaka ya Maonesho ya Viw...,1
4,"LONDON, ENGLAND BAADA ya timu ya Chelsea kula...",2


In [47]:
df['label'].value_counts()

1    10948
2     6490
4     2367
3     2005
0     1405
5      790
Name: label, dtype: int64

In [48]:
df = df[df['label'].isin([1, 2])]
df['label'].value_counts()

1    10948
2     6490
Name: label, dtype: int64

In [49]:
print(df['text'].iloc[2])

 MKURUGENZI Mkuu wa Mamlaka ya Maonesho ya Viwanda na Biashara (Tantrade), Edwin Rutageruka ameiomba Serikali kuwatumia vyema maofisa biashara wa halmashauri za wilaya, kwa kuwa wao ndio wabebaji wakuu wa ajenda ya Rais John Magufuli ya kuijenga Tanzania ya Viwanda.Aidha, amewaahidi wakazi wa mkoa huo wa Tabora kuwa atatumia fursa aliyonayo, kutangaza masoko ya bidhaa wanazozalisha ikiwemo mpunga, tumbaku, asali na alizeti ikiwa ni pamoja na kuwashauri kulima kisasa kulingana na mahitaji ya soko.Mkurugenzi huyo alisema hayo mjini Tabora jana wakati akizungumza kwenye jukwaa la biashara la mkoa wa Tabora, lilialoandaliwa na Kampuni ya Magazeti (TSN) kwa kushirikiana na mkoa wa Tabora, lenye lengo la kutangaza fursa za uwekezaji na biashara za mkoa huo. Alisema kwa sasa maofisa hao, hawatumiki ipasavyo katika maeneo yao ya wilaya zaidi ya kukusanya maduhuli katika mahoteli, masoko na maeneo mengine.“Hawa ndio wanatakiwa kuibeba hasa ajenda ya Rais ya Uchumi wa Viwanda, naomba serikali iw

In [32]:
df['text']=df['text'].str.replace("@","")
df['text']=df['text'].str.replace("#","")
df['text']=df['text'].str.replace("\d+","")
df['text']=df['text'].str.lower()

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 1))
tfidf.fit(df['text'].values.astype('U'))
tfidf = tfidf.transform(df['text'].values.astype('U'))
print(tfidf.shape)


(17438, 170359)


In [62]:
from sklearn.model_selection import train_test_split as ts

Xt,Xts,yt,yts=ts(tfidf,df['label'])
from sklearn.neighbors import KNeighborsClassifier


knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(Xt,yt)
knn_pred=knn.predict(Xts)
knn.score(Xts, yts)


0.9639908256880734

In [75]:
from sklearn.metrics import confusion_matrix
a=confusion_matrix(yts, knn_pred)
a

array([[2623,  111],
       [  46, 1580]])

In [109]:
class Metrics():
  def __init__(self,confusion_matrix):
    self.cm= confusion_matrix
    self.TP=confusion_matrix[1,1]
    self.TN=confusion_matrix[0,0]
    self.FP=confusion_matrix[0,1]
    self.FN=confusion_matrix[1,0]

    self.precision=self.TP/(self.TP+self.FP)
    self.recall=self.TP/(self.TP+self.FN)
    self.f1score=2*((self.TP/(self.TP+self.FP))*(self.TP/(self.TP+self.FN)))/(self.TP/((self.TP+self.FP))+(self.TP/(self.TP+self.FN)))
  def get_precision(self):
    return self.precision
  def get_recall(self):
    return self.recall
  def get_f1score(self):
    return self.f1score
    

In [92]:
cm=Metrics(a)
cm.get_precision()

0.9343583678296866

In [93]:
cm.get_recall()

0.971709717097171